Image Preprocessing without background removal

In [19]:
import cv2
import numpy as np
import os
import random

# Define input and output directory paths
input_base_folder = "../skinType"  # Parent folder containing 'oily', 'dry', 'normal'
output_base_folder = "./Processed_Images/"

# Fixed patch size
PATCH_SIZE = 256
MIN_SKIN_AREA_RATIO = 0.9  # Minimum 20% of the image should be skin

def detect_skin(image):
    """Detect skin using HSV and YCrCb color spaces."""
    hsv = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)
    ycrcb = cv2.cvtColor(image, cv2.COLOR_BGR2YCrCb)
    
    # Define skin color range in HSV
    lower_hsv = np.array([0, 20, 70], dtype=np.uint8)
    upper_hsv = np.array([20, 255, 255], dtype=np.uint8)
    
    # Define skin color range in YCrCb
    lower_ycrcb = np.array([0, 133, 77], dtype=np.uint8)
    upper_ycrcb = np.array([255, 173, 127], dtype=np.uint8)
    
    # Apply masks
    mask_hsv = cv2.inRange(hsv, lower_hsv, upper_hsv)
    mask_ycrcb = cv2.inRange(ycrcb, lower_ycrcb, upper_ycrcb)
    
    # Combine masks
    combined_mask = cv2.bitwise_and(mask_hsv, mask_ycrcb)
    
    # Apply Otsu thresholding
    _, otsu_mask = cv2.threshold(combined_mask, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)
    
    # Morphological operations to remove noise
    kernel = np.ones((5, 5), np.uint8)
    clean_mask = cv2.morphologyEx(otsu_mask, cv2.MORPH_CLOSE, kernel)
    
    return clean_mask

def extract_skin(image, mask):
    """Find the largest skin region, ensure it meets the minimum skin area threshold."""
    contours, _ = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    
    if not contours:
        return None  # No skin detected
    
    # Find the largest skin region
    largest_contour = max(contours, key=cv2.contourArea)
    area = cv2.contourArea(largest_contour)
    
    # Calculate image size
    img_height, img_width = image.shape[:2]
    total_pixels = img_height * img_width
    
    # Ignore images where skin area is less than 20% of total pixels
    if area / total_pixels < MIN_SKIN_AREA_RATIO:
        return None
    
    # Get bounding box of skin region
    x, y, w, h = cv2.boundingRect(largest_contour)
    
    # Crop skin region
    skin_crop = image[y:y+h, x:x+w]
    
    # Ensure minimum size for cropping
    if w < PATCH_SIZE or h < PATCH_SIZE:
        return None  # Skip small regions
    
    return skin_crop

def extract_patches(skin_region):
    """Generate a random number of patches from the skin region."""
    patches = []
    h, w = skin_region.shape[:2]
    
    # Randomly choose the number of patches (between 2 to 6)
    num_patches = random.randint(2, 6)
    
    for _ in range(num_patches):
        # Select random starting points for patches
        x_start = random.randint(0, max(1, w - PATCH_SIZE))
        y_start = random.randint(0, max(1, h - PATCH_SIZE))
        
        patch = skin_region[y_start:y_start + PATCH_SIZE, x_start:x_start + PATCH_SIZE]
        
        # Ensure the patch is of correct size before adding
        if patch.shape[0] == PATCH_SIZE and patch.shape[1] == PATCH_SIZE:
            patches.append(patch)
    
    return patches

# Process each category folder (oily, dry, normal)
for category in ["oily", "dry", "normal"]:
    input_folder = os.path.join(input_base_folder, category)
    output_folder = os.path.join(output_base_folder, category)
    os.makedirs(output_folder, exist_ok=True)
    
    for filename in os.listdir(input_folder):
        if filename.lower().endswith(('.png', '.jpg', '.jpeg')):
            img_path = os.path.join(input_folder, filename)
            image = cv2.imread(img_path)
            
            if image is None:
                continue  # Skip unreadable images
            
            skin_mask = detect_skin(image)
            skin_region = extract_skin(image, skin_mask)
            
            if skin_region is not None:  # Save only if enough skin is detected
                patches = extract_patches(skin_region)
                for i, patch in enumerate(patches):
                    output_path = os.path.join(output_folder, f"{filename.split('.')[0]}_patch{i}.jpg")
                    cv2.imwrite(output_path, patch)

print("Processing complete. Only valid skin images saved.")


Processing complete. Only valid skin images saved.


Image preprocessing with background removal

In [12]:
import cv2
import numpy as np
import os

input_base_folder = "../skinType"  # Parent folder containing oily, dry, normal
output_base_folder = "./Processed Images/"

# Fixed patch size
PATCH_SIZE = 128
NUM_PATCHES = 4  # Maximum patches per image
MIN_SKIN_AREA_RATIO = 0.4  # Minimum 40% of the image should be skin

def detect_skin(image):
    """Detects skin using HSV and YCrCb color filtering and removes background."""
    hsv = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)
    ycrcb = cv2.cvtColor(image, cv2.COLOR_BGR2YCrCb)
    
    # Define skin color range in HSV
    lower_hsv = np.array([0, 20, 70], dtype=np.uint8)
    upper_hsv = np.array([20, 255, 255], dtype=np.uint8)
    
    # Define skin color range in YCrCb
    lower_ycrcb = np.array([0, 133, 77], dtype=np.uint8)
    upper_ycrcb = np.array([255, 173, 127], dtype=np.uint8)
    
    # Apply masks
    mask_hsv = cv2.inRange(hsv, lower_hsv, upper_hsv)
    mask_ycrcb = cv2.inRange(ycrcb, lower_ycrcb, upper_ycrcb)
    
    # Combine both masks
    combined_mask = cv2.bitwise_and(mask_hsv, mask_ycrcb)
    
    # Apply Otsu thresholding
    _, otsu_mask = cv2.threshold(combined_mask, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)
    
    # Morphological operations to remove noise
    kernel = np.ones((5, 5), np.uint8)
    clean_mask = cv2.morphologyEx(otsu_mask, cv2.MORPH_CLOSE, kernel)
    
    return clean_mask

def extract_skin(image, mask):
    """Finds and extracts the largest skin region, removing background."""
    contours, _ = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    
    if not contours:
        return None  # No skin detected
    
    # Find the largest skin region
    largest_contour = max(contours, key=cv2.contourArea)
    area = cv2.contourArea(largest_contour)
    
    # Calculate image size
    img_height, img_width = image.shape[:2]
    total_pixels = img_height * img_width
    
    # Ignore images where skin area is less than 40% of total pixels
    if area / total_pixels < MIN_SKIN_AREA_RATIO:
        return None
    
    # Get bounding box of skin region
    x, y, w, h = cv2.boundingRect(largest_contour)
    
    # Create a mask with only the detected skin region
    skin_only_mask = np.zeros_like(mask)
    cv2.drawContours(skin_only_mask, [largest_contour], -1, (255), thickness=cv2.FILLED)
    
    # Apply mask to remove background
    skin_image = cv2.bitwise_and(image, image, mask=skin_only_mask)
    
    # Crop to the bounding box
    skin_crop = skin_image[y:y+h, x:x+w]
    
    # Ensure minimum size for cropping
    if w < PATCH_SIZE or h < PATCH_SIZE:
        return None  # Skip small regions
    
    return skin_crop

def extract_patches(skin_region):
    """Generates patches from the skin region."""
    patches = []
    h, w = skin_region.shape[:2]
    
    step_x, step_y = w // 2, h // 2  # Divide into 4 patches
    
    for i in range(2):
        for j in range(2):
            x_start, y_start = i * step_x, j * step_y
            patch = skin_region[y_start:y_start + step_y, x_start:x_start + step_x]
            patch_resized = cv2.resize(patch, (PATCH_SIZE, PATCH_SIZE))
            patches.append(patch_resized)
    
    return patches[:NUM_PATCHES]  # Return limited patches

# Process each category folder (oily, dry, normal)
for category in ["oily", "dry", "normal"]:
    input_folder = os.path.join(input_base_folder, category)
    output_folder = os.path.join(output_base_folder, category)
    os.makedirs(output_folder, exist_ok=True)
    
    for filename in os.listdir(input_folder):
        if filename.lower().endswith(('.png', '.jpg', '.jpeg')):
            img_path = os.path.join(input_folder, filename)
            image = cv2.imread(img_path)
            
            if image is None:
                continue  # Skip unreadable images
            
            skin_mask = detect_skin(image)
            skin_region = extract_skin(image, skin_mask)
            
            if skin_region is not None:  # Save only if enough skin is detected
                patches = extract_patches(skin_region)
                for i, patch in enumerate(patches):
                    output_path = os.path.join(output_folder, f"{filename.split('.')[0]}_patch{i}.jpg")
                    cv2.imwrite(output_path, patch)

print("Processing complete. Only valid skin images saved with background removed.")


Processing complete. Only valid skin images saved with background removed.
